In [ ]:
# Load keys
loadkeys us

In [ ]:
# Wifi
iwctl
    station wlan0 connect [WiFi Network SSID]
# It will ask for password
exit

In [ ]:
# Examine and partition the disk
fdisk -l
wipefs -a /all disk partitions 
wipefs -a /disk

# Disk layout
gdisk /disk
    # Leave 2M unused at beginning and 2G unused at end
    o # create GPT disk
    n # create each partition
    w # save

# EFI 512M type=ef00
mkfs.vfat /part1

# Swap (maybe) 512M type=8200
##mkswap /partwhatever
##swapon /partwhatever

# I am skipping swap in favor of zramd

# Timeshift part Part 50GB type=8300
mkfs.ext4 /part2

# Main Partition ~550GB type=8300
mkfs.ext4 /part3

# Consider leaving 200-300GB at end for a security-focused partition
### To boot into Kali and Tails USB and use Virtual Machines

## Make sure to leave space (2GB) at the end, good practice for SSD

In [ ]:
# Mounting
mount /part3 /mnt
mkdir -p /mnt/boot/efi
mount /part1 /mnt/boot/efi
mkdir -p /mnt/timeshift
mount /part2 /mnt/timeshift

In [ ]:
# Refresh Arch mirrors

# Check mirrorlist if you want
cat /etc/pacman.d/mirrorlist

# Update mirrorlist
pacman -Syyy

In [ ]:
# pacstrap base
pacstrap /mnt base base-devel linux linux-firmware git vim intel-ucode

In [ ]:
# Create fstab
genfstab -U /mnt >> /mnt/etc/fstab

# Make sure fstab looks alright
cat /mnt/etc/fstab

In [ ]:
# Arch-chroot into fresh install
arch-chroot /mnt

In [ ]:
# Can either type line-by-line or chmod +x to execute after editing

#!/bin/bash

# Locales
ln -sf /usr/share/zoneinfo/America/New_York /etc/localtime
hwclock --systohc
sed -i '177s/.//' /etc/locale.gen
locale-gen
echo "LANG=en_US.UTF-8" >> /etc/locale.conf
echo "KEYMAP=us" >> /etc/vconsole.conf

# Host name is computer in this script
echo "computer" >> /etc/hostname
echo "127.0.0.1 localhost" >> /etc/hosts
echo "::1       localhost" >> /etc/hosts
echo "127.0.1.1 computer.localdomain computer" >> /etc/hosts
echo yourrootpassword | passwd

pacman -S grub efibootmgr dosfstools mtools os-prober linux-headers networkmanager network-manager-applet wpa_supplicant wireless_tools dialog
pacman -S xdg-user-dirs xdg-utils xdg-desktop-portal-gtk archlinux-xdg-menu fakeroot
# Install xorg stuff
pacman -S xorg xorg-server xorg-xinit xorg-xhost xorg-xinput xorg-xkill xorg-xprop xorg-xrandr autorandr xorg-xsetroot xorg-xwininfo xorg-xkbutils xorg-xprop xdotool xorg-xmodmap xorg-x11perf xorg-xauth xorg-iceauth 
pacman -S pacman-contrib automake iptables-nft nfs-utils nilfs-utils inetutils dnsutils bridge-utils dnsmasq ntp numlockx poppler-glib
pacman -S acpi acpi_call acpid openbsd-netcat ipset nss-mdns tumbler hdparm laptop-detect less
pacman -S gvfs gvfs-smb gvfs-nfs gvfs-afc input-devices-support dconf-editor exfatprogs file-roller
pacman -S reflector tlp wget2 flatpak pcmanfm vim-plugins usbutils firewalld bash-completion rsync openssh ntfs-3g dmraid unzip unrar gzip
pacman -S mesa xf86-video-intel nvidia nvidia-settings nvidia-utils bluez bluez-utils blueberry
pacman -S sof-firmware alsa-firmware alsa-utils alsa-plugins alsa-support pipewire pipewire-pulse pipewire-alsa pipewire-jack pavucontrol mpv

grub-install --target=x86_64-efi --efi-directory=/boot/efi --bootloader-id=GRUB
grub-mkconfig -o /boot/grub/grub.cfg

reflector
systemctl enable NetworkManager
systemctl enable fstrim.timer
#systemctl enable reflector.timer
systemctl enable firewalld
#systemctl enable acpid
#systemctl enable tlp
#systemctl enable avahi-deamon

# Use your actual username instead of username
useradd -m -g users -G wheel username
echo youruserpassword | passwd username

# Edit sudoers
echo "%wheel ALL=(ALL) ALL" >> visudo

In [ ]:
# Make sure it works

# Exit chroot env
exit

# Umount
umount -a
# Reboot
reboot

# Log in as user and select network
nmtui 

In [ ]:
# Add multilib
sudo vim /etc/pacman.conf
    # Uncomment multilib repo
    
# Save mirrorlist config
sudo reflector --country US --protocol https --latest 10 --age 12 --sort rate --save /etc/pacman.d/mirrorlist
sudo systemctl enable --now reflector.timer
# Update
sudo pacman -Syu

In [ ]:
# Update mkinitcpio
sudo vim /etc/mkinitcpio.conf
    # add MODULES(i915 nvidia)

sudo mkinitcpio -p linux

# In etc/fstab
# Make sure root partition has discard option for good usage

In [ ]:
# Get AUR helper
mkdir ~/git-installs
cd ~/git-installs
git clone https://aur.archlinux.org/yay.git
cd yay
makepkg -si

In [ ]:
# Install AUR packages
yay -S optimus-manager optimus-manager-qt
yay -S timeshift stacer
yay -S rustup
 # then run rustup default stable
yay -S brave-bin libreoffice-fresh 
yay -S zramd
yay -S mullvad-vpn
yay -S lightdm lightdm-gtk-greeter lightdm-gtk-greeter-settings polkit
yay -S lxsession lxappearance kvantum-qt5 qt5ct yad scrot brightnessctl gtk-engine-murrine ant-dracula-gtk-theme ant-dracula-kvantum-theme
yay -S picom-ibhagwan-git qtile
yay -S visual-studio-code-bin virtualbox etcher-bin upower
yay -S nwg-launchers dmenu rofi kitty urxvt nitrogen xarchiver
yay -S awesome-terminal-fonts ttf-jetbrains-mono ttf-anonymous-pro ttf-ms-fonts ttf-roboto-mono ttf-joypixels ttf-font-awesome ttf-hack ttf-ubuntu-font-family ttf-hellvetica adobe-source-code-pro-fonts
yay -S isw #nvidia-settings may already have fans feature
yay -S jupyter-notebook downgrade appimagelauncher gparted htop lshw cpu-x inxi
yay -S python-extras python-psutil 

In [ ]:
# Configure and enable services

# Git clone your repos
#Follow the fan config

# Swap
sudo vim /etc/default/zramd.conf
    # changes: MAX=1500 PRIORITY=20 FRACTION=0.2
sudo systemctl enable --now zramd

# Audio 
systemctl --user enable pipewire pipewire-pulse pipewire-media-session

# Optimus
sudo systemctl enable optimus-manager

# LightDM
sudo systemctl enable lightdm

In [ ]:
sudo vim /etc/default/grub
    # Add GRUB_DISABLE_OS_PROBER=false
    # Add the ecsys/isw/quietsplash etc modules
sudo grub-mkconfig -o /boot/grub/grub.cfg

In [ ]:
# Remember to clone repos, edit configs, and make executable
    # .xinitrc , .config files, bash_profile, any autostarts
    # lxappearance, kvantum, qt5ct, gtk2
    
# Verify trim-support with lsblk --TRIM
# Read improving performance and fstrim/Solid State Drive pages of Arch wiki